# stf 비디오 생성 예제

In [1]:
import stf

# 설정

In [2]:
from pathlib import Path

verbose= False
slow_write = True # False

wav_std=True 
wav_std_ref_wav='./220105/release_220105/ref_audio.wav'
config_path = './220105/release_220105/front_config.json'
reference_face = './220105/release_220105/hunet.png'
front_checkpoint_path = f'./220105/release_220105/front_1213_1140_105.pth'
side_checkpoint_path = f'./220105/release_220105/side_1214_1245_039.pth'
video_start_offset_frame = 0 # 0이상 숫자: video 해당 frame부터 합성 / None: 맨 마지막에 맞춰서 합성

work_root_path = './stf_api_temp_root'
device = 'cuda:1'

# 템플릿 비디오 경로
front_template_video_path = './220105/demo_src/Est Soft 2_1.0-80.mov' # 보라색 자켓, 고개 끄덕 템플릿 (고개 간혹 끄덕임 주문했으나 거의 정자세)
side_template_video_path = './220105/demo_src/Est Soft 2_2.0-80.mov' # 보라색 자켓, 고개 끄덕 템플릿 (고개 간혹 끄덕임 주문했으나 거의 정자세)

# wav 경로
wav_path = '220105/demo_src/hunet_only_voice_test_50000.wav'


In [3]:
import os
assert(os.path.exists(config_path))
assert(os.path.exists(reference_face))
assert(os.path.exists(front_template_video_path))
assert(os.path.exists(front_checkpoint_path))
assert(os.path.exists(side_template_video_path))
assert(os.path.exists(side_checkpoint_path))

# 1. 템플릿 비디오 전처리

In [4]:
%%time 
def process_callback(per: int) -> None:
    print(per)
    
# template video 전처리 : 처음 실행되었을 때만 실제로 전처리가 발생하고, 이 후 여러번 실행시키면 전처리 되어있는지 확인과정만 일어난다.
stf.preprocess_template(config_path, front_template_video_path,
                        reference_face=reference_face,
                        work_root_path=work_root_path,
                        call_back=process_callback,
                        device=device,
                        verbose=verbose)
stf.preprocess_template(config_path, side_template_video_path,
                        reference_face=reference_face,
                        work_root_path=work_root_path,
                        call_back=process_callback,
                        device=device,
                        verbose=verbose)

load MTCNN  success ^ ^
load g_recognizer  success ^ ^
CPU times: user 3.49 s, sys: 1.38 s, total: 4.88 s
Wall time: 6.48 s


# 2. 팁러닝 모델 로드

In [5]:
%%time 
front_model = stf.create_model(config_path, front_checkpoint_path, work_root_path, device=device, verbose=verbose)

CPU times: user 1.71 s, sys: 511 ms, total: 2.22 s
Wall time: 2.31 s


# 3. 템플릿 비디오 읽기

In [6]:
%%time 
front_template = stf.template_video(front_model, front_template_video_path, callback=None, verbose=verbose)

CPU times: user 9.25 s, sys: 32.4 s, total: 41.7 s
Wall time: 1min 29s


# 4. 비디오 생성

In [7]:
%%time 
from pathlib import Path
%pdb on
name_prefix=''
out_path = 'temp_result/' + f'{name_prefix}_#video#-{Path(front_template_video_path).stem}_#wav#-{Path(wav_path).stem}_#weight#-{Path(front_checkpoint_path).stem}.mp4'
front_template.gen(wav_path, wav_std, wav_std_ref_wav,
                   video_start_offset_frame=video_start_offset_frame,
                   out_path=out_path,
                   slow_write=slow_write)

Automatic pdb calling has been turned ON
mel_step_size:108, mel_ps:80
CPU times: user 1min 18s, sys: 1min 14s, total: 2min 33s
Wall time: 7min 6s


'temp_result/_#video#-Est Soft 2_1.0-80_#wav#-hunet_only_voice_test_50000_#weight#-front_1213_1140_105.mp4'

# 5. 생성된 비디오 보기

In [8]:
import IPython.display as ipd
ipd.Video(out_path, width=1280)

In [9]:
# 템플릿 비디오 삭제
print('del template')
del front_template

# 모델 삭제
print('del model')
del front_model


del template
del model
